In [1]:
!pip --version

# To install pytorch -------------------------------------------------------------------------------------
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# To install Tensorflow ----------------------------------------------------------------------------------
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

# To install Flax ----------------------------------------------------------------------------------------
!pip install flax

# or to install the latest version of Flax:
!pip install --upgrade git+https://github.com/google/flax.git

# Set up transformers
!pip install git+https://github.com/huggingface/transformers

# Check if installed
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I love you'))"

!pip install transformers[torch]

!pip install accelerate -U

!pip install -q datasets peft evaluate

!pip install opacus

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/google/flax.git to /tmp/pip-req-build-j_5dy4qu
  Running command git clone --filter=blob:none --quiet https://github.com/google/flax.git /tmp/pip-req-build-j_5dy4qu
  Resolved https://github.com/google/flax.git to commit 85dfad242e56098849dbf05e7e4657b3a40820f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for flax: filename=flax-0.8.0-py3-none-any.whl size=635291 sha256=74542bcb5969480ed867be656b65a3f77e349c6d9fce1edb551411e7f41c9324
  Stored in directory: /tmp/p

In [2]:
from transformers import (AutoModelForSequenceClassification,AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer )
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
    PromptTuningConfig,
)
from peft import ( get_peft_config, get_peft_model, get_peft_model_state_dict, set_peft_model_state_dict, PeftModelForSeq2SeqLM, PeftType,PromptEncoderConfig,PeftModelForSequenceClassification)
from peft import PromptEmbedding, PromptTuningConfig
from datasets import load_dataset
import evaluate
from transformers import AdamW
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from datasets import Dataset
import opacus
from opacus import PrivacyEngine
from opacus.grad_sample import GradSampleModule
from opacus.validators import ModuleValidator



dataset = load_dataset("glue", "qqp")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=3) # as output 0 or 1
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny", padding_side = "right")
model = model.to('cuda')
# optimizer = AdamW(model.parameters(), lr= 0.005, eps = 1e-8)


Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [3]:
config = PromptTuningConfig(
    peft_type="PROMPT_TUNING",
    task_type="TEXT_PAIR",
    num_virtual_tokens= 50,
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="Classify the given question and sentence pair as entailment, neutral or contradiction",
    tokenizer_name_or_path="prajjwal1/bert-tiny",
)

model = PeftModelForSequenceClassification(model, config)
# model = get_peft_model(model, config)
print(model.print_trainable_parameters())

total_params = 0
trainable_params = 0

trainable_layers = [model.prompt_encoder, model.classifier]
for p in model.parameters():
        p.requires_grad = False
        total_params += p.numel()

l = []
for layer in trainable_layers:
    for p in layer.parameters():
        l.append(p)
        p.requires_grad = True
        trainable_params += p.numel()

optimizer = AdamW(l , lr= 0.005, eps = 1e-8)

print("total:",total_params)
print("trainable_param",trainable_params)


trainable params: 6,787 || all params: 4,393,094 || trainable%: 0.15449248297441392
None
total: 4393094
trainable_param 7174


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
config = PromptTuningConfig(
    peft_type="PROMPT_TUNING",
    task_type="TEXT_PAIR",
    num_virtual_tokens= 7,
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="Classify the question-sentence pair as entailment, neutral or contradiction",
    tokenizer_name_or_path="prajjwal1/bert-tiny",
)

# peft_config = PromptTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=60)
# model = get_peft_model(model, peft_config)


model = PeftModelForSequenceClassification(model, config)
model = get_peft_model(model, config)
print(model.print_trainable_parameters())


model = ModuleValidator.fix(model)
model = GradSampleModule(model)

optimizer = AdamW(model.parameters() , lr= 0.05, eps = 1e-8)



In [4]:
# Assuming the dataset has a 'train' split, modify this according to your dataset's splits
data_split = dataset['train']

# Convert the dataset split to a pandas DataFrame for easier splitting
df = data_split.to_pandas()
df = df[0:150000]

# Split the dataset into train and test sets using train_test_split from sklearn
# df1, df2 = train_test_split(df, test_size = 0.5, random_state = 42)
train_df, test_df = train_test_split(df , test_size=0.2, random_state=42)

# Convert the splits back to datasets
train_dataset = train_df.reset_index(drop=True)
test_dataset = test_df.reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

x_train1 = list(train_dataset["question1"])
x_train2 = list(train_dataset["question2"])
y_train = list(train_dataset["label"])

x_test1 = list(test_dataset["question1"])
x_test2 = list(test_dataset["question2"])
y_test = list(test_dataset["label"])

X_train_tokenized = tokenizer(x_train1, x_train2, padding=True, truncation=True, max_length = 128)
# X_val_tokenized = tokenizer(x_validation, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(x_test1, x_test2, padding=True, truncation = True, max_length = 128)


class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to('cuda') for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx]).to('cuda')
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
# val_dataset = Dataset(X_val_tokenized, y_validation)
test_dataset = Dataset(X_test_tokenized, y_test)

train_dataloader = DataLoader(train_dataset, batch_size = 1024, drop_last = True)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size = 1024, drop_last = True)
# val_dataloader = DataLoader(val_dataset, batch_size = 500)

In [ ]:
model.train()
privacy_engine = PrivacyEngine()
model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    target_delta= 1 / len(train_dataloader),
    target_epsilon=  8,
    epochs= 3,
    max_grad_norm = 0.05,
)


In [5]:
def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()
    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:

        with torch.no_grad():
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss, logits = outputs[:2]

            log = logits.cpu()
            log = log.numpy()

            preds = np.argmax(log, axis=1)
            labels = batch['labels'].detach().cpu().numpy()

            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

model.train()

# Training loop
optimizer.zero_grad()  # Explicitly zero the gradient buffers

for epoch in range(60):  # Number of epochs - 10 in privacy
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

        logits = outputs[1]
        predictions = torch.argmax(logits, dim = -1)

    eval_loss, eval_accuracy = evaluate(model)
    print("eval loss",eval_loss)
    print("accuracy: ",eval_accuracy)
    print("ends")

    model.train()  # Set the model back to training mode


eval loss 0.6125419920888441
accuracy:  0.656182650862069
ends
eval loss 0.5851464189332107
accuracy:  0.6833580280172413
ends
eval loss 0.5526887404507604
accuracy:  0.7033943965517241
ends
eval loss 0.5452784895896912
accuracy:  0.7063914331896551
ends
eval loss 0.5375146064265021
accuracy:  0.712890625
ends
eval loss 0.5233394791340006
accuracy:  0.7198275862068966
ends
eval loss 0.5223659153642326
accuracy:  0.7227572737068966
ends
eval loss 0.5136934272174177
accuracy:  0.7261920797413793
ends
eval loss 0.5117143228136259
accuracy:  0.7302667025862069
ends
eval loss 0.5078718734198603
accuracy:  0.7301656788793104
ends
eval loss 0.5102031045946581
accuracy:  0.7275053879310345
ends
eval loss 0.5041940530826305
accuracy:  0.7342066271551724
ends
eval loss 0.49946315946250125
accuracy:  0.7381465517241379
ends
eval loss 0.503269520299188
accuracy:  0.7349474676724138
ends
eval loss 0.4985828985427988
accuracy:  0.7379781788793104
ends
eval loss 0.501006117154812
accuracy:  0.7377087

KeyboardInterrupt: ignored